In [2]:
!pip3 install ultralytics

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.


In [3]:
from ultralytics import YOLO
from IPython.display import display, clear_output
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import ipywidgets as widgets
#Import packages for this bih

In [4]:
# Load pretrained YOLO
model = YOLO('yolov8x.pt')

In [5]:
cap = cv2.VideoCapture('./pitching videos/IMG_3275.mov') #Read in pitch from folder
frame_list = []  #List where we will store each frame of the video
while(cap.isOpened()): 
      
# Capture frame-by-frame 
    ret, frame = cap.read() 
    if ret == True: 
    # Display the resulting frame 
        # cv2.imshow('Frame', frame) 
        frame_list.append(frame)
    # Press Q on keyboard to exit 
        # if cv2.waitKey(25) & 0xFF == ord('q'): 
        #     break
    else: 
        break

# the video capture object 
cap.release()

# Closes all the frames 
cv2.destroyAllWindows()
cv2.waitKey(1)

print(len(frame_list))
print(frame_list[0].shape[0])
print(frame_list[0].shape[1])

191
1584
2816


In [6]:
cv2.imshow('Frame', frame_list[40])
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 
cv2.waitKey(1)

-1

In [7]:
CANVAS_SIZE = (frame_list[0].shape[0], frame_list[0].shape[1])  # height, width

FINAL_LINE_COLOR = (0, 0, 255)
WORKING_LINE_COLOR = (127, 127, 127)
THICKNESS = 3
# ============================================================================

class PolygonDrawer(object):
    def __init__(self, window_name,canvas):
        self.window_name = window_name # Name for our window
        self.canvas = np.copy(canvas)
        self.done = False # Flag signalling we're done
        self.current = (0, 0) # Current position, so we can draw the line-in-progress
        self.points = [] # List of points defining our polygon
#         self.working_points = [] #Points that will show where the cursor currently is, but will not be saved into final image

    def on_mouse(self, event, x, y, buttons, user_param):
        # Mouse callback that gets called for every mouse event (i.e. moving, clicking, etc.)

        if self.done: # Nothing more to do
            return

        if event == cv2.EVENT_MOUSEMOVE:
            # We want to be able to draw the line-in-progress, so update current mouse position
            self.current = (x, y)
        elif event == cv2.EVENT_LBUTTONDOWN:
            # Left click means adding a point at current position to the list of points
            print("Adding point #%d with position(%d,%d)" % (len(self.points), x, y))
            self.points.append((x, y))
        elif event == cv2.EVENT_RBUTTONDOWN or len(self.points) == 2:
            # Right click means we're done
            print("Completing polygon with %d points." % len(self.points))
            self.done = True


    def run(self):
        # Let's create our working window and set a mouse callback to handle events
        cv2.namedWindow(self.window_name, flags=cv2.WINDOW_NORMAL)
        cv2.imshow(self.window_name, self.canvas)
        cv2.waitKey(1)
        cv2.setMouseCallback(self.window_name, self.on_mouse)
        clean = np.copy(self.canvas)
        while(not self.done):
            # This is our drawing loop, we just continuously draw new images
            # and show them in the named window
            #For our purposes, we are going to select the top left, and bottom right corners of  the 
            #strike zone and then connect the points together in a rectangle
                                   
            if (len(self.points) > 0):
                # Draw all the current polygon segments
#                 cv2.polylines(self.canvas, np.array([self.points]), False, FINAL_LINE_COLOR, 1)
                cv2.circle(self.canvas, self.points[-1], 5, FINAL_LINE_COLOR, THICKNESS)
                # And  also show what the current segment would look like
#                 cv2.line(self.canvas, self.points[-1], self.current, WORKING_LINE_COLOR,2)
            # Update the window
            cv2.imshow(self.window_name, self.canvas)
            # And wait 50ms before next iteration (this will pump window messages meanwhile)
            if cv2.waitKey(50) == 27: # ESC hit
                self.done = True

        # User finised entering the polygon points, so let's make the final drawing
        self.canvas = np.copy(clean) #Clean copy of the canvas (no lines) which will get returned to the user
#         of a filled polygon
        if (len(self.points) > 0):
            tl = self.points[0]
            br = self.points[1]
            bl = (tl[0],br[1])
            tr = (br[0],tl[1])
            strike_zone_top = [tl,tr]
            strike_zone_right = [tr,br]
            strike_zone_bottom = [br,bl]
            strike_zone_left = [bl,tl]
            
            cv2.polylines(self.canvas, np.array([strike_zone_top]),False, FINAL_LINE_COLOR,THICKNESS)
            cv2.polylines(self.canvas, np.array([strike_zone_right]),False, FINAL_LINE_COLOR,THICKNESS)
            cv2.polylines(self.canvas, np.array([strike_zone_bottom]),False, FINAL_LINE_COLOR,THICKNESS)
            cv2.polylines(self.canvas, np.array([strike_zone_left]),False, FINAL_LINE_COLOR,THICKNESS)
# #         And show it
        cv2.imshow(self.window_name, self.canvas)
        # Waiting for the user to press any key
        cv2.waitKey()

        cv2.destroyWindow(self.window_name)
        cv2.waitKey(1)
        zone = self.canvas - clean #Subtract off difference between original image and marked up image
        
        zone_r = zone[:,:,2] #break into 3 color components r,g,b
        zone_g = zone[:,:,1]
        zone_b = zone[:,:,0]
        

        zone_r[zone_r!=0] = FINAL_LINE_COLOR[2] #Find where these are not equal to zero and set them to the color of the line
                                                #used for marking up
        zone_g[zone_g!=0] = FINAL_LINE_COLOR[1]
        zone_b[zone_b!=0] = FINAL_LINE_COLOR[0]
        
        zone_line = np.stack((zone_b,zone_g,zone_r)).T.swapaxes(0,1) #stack em back up
        
        return self.canvas,zone_line


In [ ]:
cv2.imshow('Frame',frame_list[0]) #Show random frame of the video. I picked 200 to do my editing on but it doesn't matter
print(np.shape(frame_list[0].swapaxes(0,1)))
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 
cv2.waitKey(1)


In [9]:
if __name__ == "__main__":
    canvas = np.copy(frame_list[0])
    pd = PolygonDrawer("Polygon",canvas)
    image,zone_outline = pd.run()
    cv2.imwrite("polygon.png", image)
    print("Polygon = %s" % pd.points)


Adding point #0 with position(947,744)
Adding point #1 with position(1423,1248)
Completing polygon with 2 points.
Polygon = [(947, 744), (1423, 1248)]


In [10]:
# get the strike zone dimensions with respect to the canvas
top_left_point = pd.points[0]
bottom_right_point = pd.points[1]

zone_x_start = top_left_point[0]
zone_x_end = bottom_right_point[0]
zone_y_start = top_left_point[1]
zone_y_end = bottom_right_point[1]

ball_width_start = 20
ball_width_end = 300

print(f"The x pixel range for the strike zone is {zone_x_start} to {zone_x_end}.")
print(f"The y pixel range for the strike zone is {zone_y_start} to {zone_y_end}.")

The x pixel range for the strike zone is 947 to 1423.
The y pixel range for the strike zone is 744 to 1248.


In [11]:
def is_strike(x1, y1, x2, y2):
    ball_pixel_width = x2 - x1
    if ball_pixel_width > ball_width_start:
        # ball is within strike zone range, check if it is in the 2D strike zone or not
        mid_y = y1 + (y2 - y1) / 2
        mid_x = x1 + (x2 - x1) / 2
        if ((mid_x > zone_x_start) and (mid_x < zone_x_end) and (mid_y > zone_y_start) and (mid_y < zone_y_end)):
            # it's a strike!
            print("Strike!")
            return True

    print("Not a strike!")
    return False

zone_r = zone_outline[:,:,2] #break up zone into r,g,b
zone_g = zone_outline[:,:,1]
zone_b = zone_outline[:,:,0]
new_frames = [] #new frames for the edited video will be stored here

strike = False
for k in range(len(frame_list)):
    # draw the strike zone for each frame in frame_list
    new_frames.append(frame_list[k])
    current_frame = new_frames[k] #select current frame
    current_frame_r = current_frame[:,:,2] #break into color components
    current_frame_g = current_frame[:,:,1]
    current_frame_b = current_frame[:,:,0]
    
    current_frame_r[zone_r != 0] = np.max(zone_r) #Find on the current frame where the zone_outline is non-zero and set to 
                                                #the max for that color channel
    current_frame_g[zone_g != 0] = np.max(zone_g)
    current_frame_b[zone_b != 0] = np.max(zone_b)

    # run the object detection on each frame then draw the bounding box + confidence on the frame
    results = model.predict(current_frame, classes=32)

    for result in results:
        print(result.boxes)
        boxes = result.boxes
        if boxes.conf.size(0) > 0:
            # There are detections
            for i in range(boxes.xyxy.size(0)): # For each given detection
                # Ensure coordinates are integers and create tuples
                x1, y1, x2, y2 = map(int, boxes.xyxy[i].tolist())
    
                # Extract confidence and class_id and convert them to native Python types if necessary
                confidence = float(boxes.conf[i])
                class_id = int(boxes.cls[i])
            
                # Draw rectangle
                cv2.rectangle(current_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

                if ((not strike) and is_strike(x1, y1, x2, y2)):
                    strike = True
                    # cv2.imshow('Frame', current_frame)
                    # cv2.waitKey(0) 
                      
                    # # closing all open windows 
                    # cv2.destroyAllWindows() 
                    # cv2.waitKey(1)
                
                # Put confidence score
                # cv2.putText(current_frame, f'{confidence*100:.1f}%', (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                # cv2.imshow('Frame', current_frame)
                # cv2.waitKey(0) 
                  
                # # closing all open windows 
                # cv2.destroyAllWindows() 
                # cv2.waitKey(1)
                break
    
    if strike:
        cv2.putText(current_frame, f'Strike!', (zone_x_start, zone_y_start-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)

if not strike:
    last_frame_index = len(new_frames) - 1
    cv2.putText(new_frames[last_frame_index], f'Ball!', (zone_x_start, zone_y_start-10), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 2)



0: 384x640 (no detections), 338.5ms
Speed: 1.7ms preprocess, 338.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([])
conf: tensor([])
data: tensor([], size=(0, 6))
id: None
is_track: False
orig_shape: (1584, 2816)
shape: torch.Size([0, 6])
xywh: tensor([], size=(0, 4))
xywhn: tensor([], size=(0, 4))
xyxy: tensor([], size=(0, 4))
xyxyn: tensor([], size=(0, 4))

0: 384x640 (no detections), 325.9ms
Speed: 1.3ms preprocess, 325.9ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([])
conf: tensor([])
data: tensor([], size=(0, 6))
id: None
is_track: False
orig_shape: (1584, 2816)
shape: torch.Size([0, 6])
xywh: tensor([], size=(0, 4))
xywhn: tensor([], size=(0, 4))
xyxy: tensor([], size=(0, 4))
xyxyn: tensor([], size=(0, 4))

0: 384x640 1 sports ball, 357.1ms
Speed: 1.3ms preprocess, 357.1ms inference, 0.7ms postpro

In [14]:
# newframes = []
#Write video to output
writer = cv2.VideoWriter("outputvideo.mp4", cv2.VideoWriter_fourcc(*"mp4v"), 30, (frame_list[0].shape[1],frame_list[0].shape[0]))
for i in range(len(new_frames)):

    writer.write(new_frames[i])  # write frame into output vid

writer.release()
print("done!")

done!


In [8]:
cv2.imshow('image',canvas) #show original
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 
cv2.waitKey(1)

-1

In [9]:
cv2.imshow('image',image)#show marked up
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 
cv2.waitKey(1)

-1

In [10]:
cv2.imshow('zone',zone_outline) #Show outline drawn by user
cv2.waitKey(0) 
  
# closing all open windows 
cv2.destroyAllWindows() 
cv2.waitKey(1)

print(np.shape(zone_outline))

(1584, 2816, 3)
